<a href="https://colab.research.google.com/github/UmarIgan/Machine-Learning/blob/master/LLMBenchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LLM Benchmarking using Cosine Similarity

In this work i benchmarked two model of LLama-2 instruct. One of them is fine-tuned by me with a financial alpaca style dataset and the other is base instruct dataset.
The benchmark done by 3 factors:


*  Cosine Similarity
*  NER Relevance
*  Keyword Score



In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline,AutoModelForSequenceClassification
import numpy as np
import spacy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import json
import re

In [2]:
dataset = [
            {
                "instruction": "Analyze the potential impact of this news on the company's stock price",
                "input": "Apple Inc. announced a breakthrough in AI chip technology, potentially revolutionizing their product lineup.",
                "context": "The news suggests a potential technological advancement that could impact Apple's market position and stock valuation."
            },
            {
                "instruction": "Explain the financial implications of this quarterly report",
                "input": "Microsoft reported a 15% increase in cloud computing revenue for Q4, exceeding analyst expectations.",
                "context": "Strong cloud computing performance indicates growth potential and positive financial momentum for Microsoft."
            },
            {
                "instruction": "Assess the economic impact of this regulatory change",
                "input": "Federal Reserve signals potential interest rate cuts in the upcoming fiscal year, aiming to stimulate economic growth.",
                "context": "Potential interest rate reductions could influence investment strategies, borrowing costs, and overall economic activity."
            },
            {
                "instruction": "Evaluate the financial potential of this startup funding round",
                "input": "AI-driven cybersecurity startup Anthropic raises $750 million in Series C funding from top-tier venture capital firms.",
                "context": "Significant funding indicates strong investor confidence in the company's technological innovation and market potential."
            },
            {
                "instruction": "Analyze the market implications of this merger",
                "input": "Google's parent company Alphabet announces acquisition of Waze for $1.3 billion, expanding its mapping and location-based services portfolio.",
                "context": "Strategic acquisitions can provide competitive advantages and open new revenue streams in the tech ecosystem."
            },
            {
                "instruction": "Predict the economic consequences of this global trade development",
                "input": "China and the United States negotiate a new trade agreement, potentially reducing tariffs on key technological and agricultural products.",
                "context": "Trade negotiations can significantly impact global supply chains, commodity prices, and international market dynamics."
            },
            {
                "instruction": "Assess the financial health of the company based on this report",
                "input": "Tesla reports reduced margins in electric vehicle sales but increases energy storage revenue by 40% in the last quarter.",
                "context": "Diversification of revenue streams and growth in alternative sectors can mitigate challenges in core business lines."
            },
            {
                "instruction": "Evaluate the investment potential of this technological breakthrough",
                "input": "Renewable energy company develops a solar panel with record-breaking 30% efficiency, potentially revolutionizing solar power generation.",
                "context": "Significant technological advancements in renewable energy can attract investment and disrupt traditional energy markets."
            },
            {
                "instruction": "Analyze the potential market shift caused by this innovation",
                "input": "Amazon launches advanced quantum computing services, targeting enterprise customers with unprecedented computational capabilities.",
                "context": "Breakthrough technologies in cloud computing and quantum technology can create new market opportunities and competitive advantages."
            },
            {
                "instruction": "Assess the financial implications of this industry trend",
                "input": "Major insurance companies invest heavily in AI-driven risk assessment and claims processing technologies.",
                "context": "Technological innovation in traditional industries can lead to increased efficiency, cost reduction, and improved customer experiences."
            },
            {
                "instruction": "Predict the economic impact of this global workforce trend",
                "input": "Remote work continues to reshape commercial real estate markets, with companies adopting hybrid work models post-pandemic.",
                "context": "Significant shifts in workplace dynamics can have far-reaching consequences for urban economies and real estate investments."
            },
            {
                "instruction": "Evaluate the financial potential of this healthcare innovation",
                "input": "Breakthrough gene therapy shows promising results for treating rare genetic disorders, with potential market applications.",
                "context": "Medical innovations can create substantial economic opportunities in biotechnology and pharmaceutical sectors."
            },
            {
                "instruction": "Analyze the market implications of this supply chain development",
                "input": "Tech companies diversify semiconductor supply chains to mitigate geopolitical risks and reduce dependency on single-source manufacturers.",
                "context": "Strategic supply chain restructuring can enhance resilience and provide competitive advantages in global technology markets."
            },
            {
                "instruction": "Assess the investment landscape in this emerging market",
                "input": "Southeast Asian countries show significant growth in digital economy, startup ecosystem, and technological innovation.",
                "context": "Emerging markets with strong digital transformation can offer attractive investment opportunities and economic potential."
            },
            {
                "instruction": "Predict the financial consequences of this regulatory change",
                "input": "Securities and Exchange Commission proposes new transparency rules for public companies, focusing on climate-related financial disclosures.",
                "context": "Enhanced regulatory requirements can impact corporate reporting, investor confidence, and market transparency."
            }
        ]


In [3]:
alpaca_model_id = "umarigan/llama3.2-1B-fin"
instruct_model_id = "meta-llama/Llama-3.2-1B-Instruct"

# Sentence embedding model for semantic similarity
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Sentiment model for NER
try:
    nlp = spacy.load('en_core_web_sm')
except OSError:
    print("Downloading spaCy English model...")
    from spacy.cli import download
    download('en_core_web_sm')
    nlp = spacy.load('en_core_web_sm')

sentiment_model = AutoModelForSequenceClassification.from_pretrained('stfamod/fine-tuned-bert-financial-sentiment-analysis')
sentiment_tokenizer = AutoTokenizer.from_pretrained('stfamod/fine-tuned-bert-financial-sentiment-analysis')

# Fine-tuned model setup
alpaca_tokenizer = AutoTokenizer.from_pretrained(alpaca_model_id)
alpaca_model = AutoModelForCausalLM.from_pretrained(alpaca_model_id)

# Instruct model setup
instruct_pipeline = pipeline(
    "text-generation",
    model=instruct_model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context.
                  Write a response that appropriately completes the request.
                  ### Instruction:
                  {}
                  ### Input:
                  {}
                  ### Response:
                  {}"""


In [4]:
def preprocess_text(text):
        """
        Preprocess text by:
        1. Removing special characters
        2. Lowercasing
        3. Removing extra whitespaces
        """
        # Remove special characters and digits
        text = re.sub(r'[^a-zA-Z\s]', '', text)
        # Lowercase and remove extra whitespaces
        return ' '.join(text.lower().split())

def semantic_similarity(response, context):
    """
    Calculate semantic similarity using sentence embeddings
    """
    # Preprocess texts
    response_processed = preprocess_text(response)
    context_processed = preprocess_text(context)

    # Generate embeddings
    response_embedding = embedding_model.encode([response_processed])
    context_embedding = embedding_model.encode([context_processed])

    # Calculate cosine similarity
    similarity = cosine_similarity(response_embedding, context_embedding)[0][0]
    return similarity
def named_entity_analysis(response):
        """
        Extract and analyze named entities in the response
        """
        doc = nlp(response)

        # Count and categorize named entities
        entity_counts = {}
        for ent in doc.ents:
            entity_counts[ent.label_] = entity_counts.get(ent.label_, 0) + 1

        return entity_counts
def financial_relevance_score(response):
        """
        Calculate financial relevance based on:
        1. Presence of financial terminology
        2. Named entity recognition
        3. Semantic context
        """
        # Financial keywords
        financial_keywords = [
            'stock', 'market', 'investment', 'revenue', 'profit', 'economic',
            'portfolio', 'earnings', 'dividend', 'capital', 'finance',
            'financial', 'trade', 'asset', 'equity', 'bond', 'futures'
        ]

        # Check keyword presence
        keyword_score = sum(
            1 for keyword in financial_keywords
            if keyword in response.lower()
        ) / len(financial_keywords)

        # Named entity analysis
        entities = named_entity_analysis(response)
        if entities:
          l = ["ORG", "MONEY", 'CARDINAL']
          s = 0
          for i in entities.keys():
            if i in l:
              s+=1
          entity_score = s / 3
        else:
          entity_score = 0
        return (keyword_score + entity_score) / 2

def generate_alpaca_response(question):
  """Generate response using Alpaca-style model"""
  inputs = alpaca_tokenizer(
            [alpaca_prompt.format(
                question['instruction'],
                question['input'],
                ""
            )],
            return_tensors="pt"
        )
  outputs = alpaca_model.generate(**inputs, max_new_tokens=128, use_cache=True)
  return alpaca_tokenizer.batch_decode(outputs)[0]

def generate_instruct_response(question):
    """Generate response using Instruct model"""
    messages = [
        {"role": "system", "content": "You are a financial analyst providing expert insights."},
        {"role": "user", "content": f"{question['instruction']}\n\n{question['input']}"}
    ]
    outputs = instruct_pipeline(
        messages,
        max_new_tokens=128,
    )
    return outputs[0]["generated_text"][-1]

In [5]:
def benchmark():
    """
    Run benchmark on both models and generate comprehensive report
    """
    alpaca_similarities = []
    instruct_similarities = []

    detailed_results = []

    for question in dataset:
        # Generate responses
        alpaca_response = generate_alpaca_response(question)
        start_marker = "### Response:"
        start_index = alpaca_response.find(start_marker) + len(start_marker)
        end_marker = "<|eot_id|>"
        end_index = alpaca_response.find(end_marker)

        # Get the response content
        alpaca_response_content = alpaca_response[start_index:end_index].strip()

        instruct_response = generate_instruct_response(question)
        # Calculate semantic similarity
        alpaca_similarity = semantic_similarity(alpaca_response_content, question['context'])
        instruct_similarity = semantic_similarity(instruct_response['content'], question['context'])

        # Calculate relevance score
        alpaca_relevance_score = financial_relevance_score(alpaca_response_content)
        instruct_relevance_score = financial_relevance_score(instruct_response['content'])

        alpaca_similarities.append(alpaca_similarity)
        instruct_similarities.append(instruct_similarity)

        detailed_results.append({
            'instruction': question['instruction'],
            'input': question['input'],
            'context': question['context'],
            'alpaca_response': alpaca_response_content,
            'instruct_response': instruct_response['content'],
            'alpaca_similarity': alpaca_similarity,
            'instruct_similarity': instruct_similarity,
            'alpaca_relevance_score':alpaca_relevance_score,
            'instruct_relevance_score':instruct_relevance_score

        })
    print("answer ends")
    # Calculate overall metrics
    return detailed_results

# Run the benchmark
results = benchmark()

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id

answer ends


In [6]:
import pandas as pd
df = pd.DataFrame(results)
df

,instruction,input,context,alpaca_response,instruct_response,alpaca_similarity,instruct_similarity,alpaca_relevance_score,instruct_relevance_score
0,Analyze the potential impact of this news on t...,Apple Inc. announced a breakthrough in AI chip...,The news suggests a potential technological ad...,Apple's AI chip technology could have a signif...,The announcement of a breakthrough in AI chip ...,0.779669,0.760238,0.225490,0.392157
1,Explain the financial implications of this qua...,Microsoft reported a 15% increase in cloud com...,Strong cloud computing performance indicates g...,Microsoft's cloud computing revenue for Q4 was...,The quarterly report from Microsoft indicates ...,0.656124,0.742032,0.254902,0.421569
2,Assess the economic impact of this regulatory ...,Federal Reserve signals potential interest rat...,Potential interest rate reductions could influ...,The potential interest rate cuts could help st...,The potential interest rate cuts by the Federa...,0.632358,0.633170,0.196078,0.392157
3,Evaluate the financial potential of this start...,AI-driven cybersecurity startup Anthropic rais...,Significant funding indicates strong investor ...,The financial potential of this startup fundin...,The significant Series C funding round of $750...,0.587389,0.577208,0.254902,0.480392
4,Analyze the market implications of this merger,Google's parent company Alphabet announces acq...,Strategic acquisitions can provide competitive...,This merger has significant implications for t...,The acquisition of Waze by Alphabet (Google's ...,0.409787,0.450371,0.254902,0.529412
5,Predict the economic consequences of this glob...,China and the United States negotiate a new tr...,Trade negotiations can significantly impact gl...,The economic consequences of this global trade...,The economic consequences of a new trade agree...,0.539716,0.478212,0.147059,0.284314
6,Assess the financial health of the company bas...,Tesla reports reduced margins in electric vehi...,Diversification of revenue streams and growth ...,"Based on this report, Tesla's financial health...","Based on the provided report, here's an assess...",0.231643,0.286764,0.254902,0.421569
7,Evaluate the investment potential of this tech...,Renewable energy company develops a solar pane...,Significant technological advancements in rene...,Renewable energy company develops a solar pane...,**Investment Potential Evaluation: Solar Panel...,0.480573,0.554519,0.000000,0.392157
8,Analyze the potential market shift caused by t...,Amazon launches advanced quantum computing ser...,Breakthrough technologies in cloud computing a...,Amazon's new quantum computing services could ...,The launch of Amazon's advanced quantum comput...,0.713012,0.684730,0.196078,0.392157
9,Assess the financial implications of this indu...,Major insurance companies invest heavily in AI...,Technological innovation in traditional indust...,This industry trend has significant financial ...,The trend of major insurance companies investi...,0.390645,0.169315,0.029412,0.196078


In [7]:
df[['alpaca_similarity', 'instruct_similarity','alpaca_relevance_score', 'instruct_relevance_score']].mean()

,0
alpaca_similarity,0.553251
instruct_similarity,0.549149
alpaca_relevance_score,0.141830
instruct_relevance_score,0.355556
